In [1]:
# import torch
# import numpy as np
# from sklearn.preprocessing import OneHotEncoder

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(torch.cuda.is_available())

# Data

In [3]:
# import os
# import pickle

# raw_file = './../../data/dump/mimic.pickle'
# with open(raw_file, 'rb') as handle:
#     data = pickle.load(handle)
# # filename = './../../data/dump/stack_processed.pickle'
# # if os.path.isfile(filename):
# #     with open(filename, 'rb') as handle:
# #         print("Processed Dictionary Exists!")
# #         data = pickle.load(handle)
# #         print(len(data))
# # else:
# #     data = process_elements(data)
# #     with open(filename, 'wb') as handle:
# #         pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
# print(type(data))
# print(len(data.keys()))

In [5]:
# for item in data:
#     for key, value in item.items():
#         print(key, value)
#         print()
#     break

In [6]:
# for key, val in data.items():
#     print(key,'\n',val)
#     print(len(val))
#     break

In [7]:
import matplotlib.pyplot as plt

## Lengths of sequences

In [8]:
# plt.hist(list(map(len, data.values())), bins=500)
# plt.xlim(0,1000)
# plt.xticks(np.linspace(0,1000,10, endpoint=True))
# plt.show()

## Convert to dataframe

In [1]:
import pandas
import numpy as np
from multiprocessing import Pool
import os
import pickle

In [2]:
import sys
sys.path.insert(0, './../utils/')

In [3]:
# base_path = './../../data/dump/data_meme/'
# files = os.listdir(base_path)
# file_list = [base_path+filename for filename in files if filename.split('.')[1]=='pkl']

In [4]:
base_path = './../../data/'
files = os.listdir(base_path)
file_list = [base_path+filename for filename in files if filename.split('.')[0].startswith('hawkes')]

In [5]:
file_list

['./../../data/hawkesdatatest.txt',
 './../../data/hawkesdataval.txt',
 './../../data/hawkesdatatrain.txt']

In [8]:
def list_to_stacked_time_array(x):
    intervals = pandas.Series(x) - pandas.Series(x).shift(1)
    intervals[0] = x[0]
    return np.stack([intervals, pandas.Series(x)]).T

def getdata_Hawkes(filepath):
    t_data, x_data = [],[]
    with open(filepath,'r') as f:
        data=f.readlines()
    # Split a line into list of floats
    data = list(map(lambda x: list(map(float, str.split(x))), data))
    t_data = list(map(list_to_stacked_time_array, data))
    x_data = list(map(lambda x: np.ones(len(x)), data))
    data_dict = {
        't': t_data,
        'x': x_data
    }
    return data_dict

In [9]:
valid_dict = getdata_Hawkes('./../../data/hawkesdataval.txt')
train_dict = getdata_Hawkes('./../../data/hawkesdatatrain.txt')
test_dict = getdata_Hawkes('./../../data/hawkesdatatest.txt')

In [13]:
list(map(len, valid_dict['t'][:5])), list(map(len, train_dict['t'][:5]))

([26, 6, 43, 41, 25], [7, 30, 27, 17, 12])

In [6]:
def seq_to_marker(seq):
    """
        Input: dict with keys: 'time_since_start', 'time_since_last_event'
    """
    seq = sorted(seq, key=lambda x: x['time_since_start'])
    markers = np.array([datum['type_event'] for datum in seq])
    return markers

def seq_to_times(seq):
    """
        Input: dict with keys: 'time_since_start', 'time_since_last_event'
    """
    seq = sorted(seq, key=lambda x: x['time_since_start'])
    times = np.array([(datum['time_since_last_event'], datum['time_since_start']) for datum in seq])
    return times

In [7]:
import random
idxs = random.sample(range(10), 5)
print(idxs)
np.arange(10)[idxs]

[5, 2, 9, 4, 8]


array([5, 2, 9, 4, 8])

In [9]:
def getdata_NHP(filepath, max_size=None):
    t_data, x_data = [],[]
    with open(filepath,'rb') as f:
        data = pickle.load(f, encoding='latin1')
        for _data in data.values():
            if (type(_data)==list) and len(_data)>0:
                x_data.extend(list(map(seq_to_marker, _data)))
                t_data.extend(list(map(seq_to_times, _data)))
    if max_size is not None:
        idxs = random.sample(range(len(x_data)), max_size)
        t_data = [t_data[idx] for idx in idxs]
        x_data = [x_data[idx] for idx in idxs]

    data_dict = {
        't': t_data,
        'x': x_data
    }
    return data_dict

In [68]:
valid_dict = getdata_NHP('./../../data/dump/data_meme/dev.pkl')
train_dict = getdata_NHP('./../../data/dump/data_meme/train.pkl')
test_dict = getdata_NHP('./../../data/dump/data_meme/test.pkl')

NameError: name 'getdata_NHP' is not defined

In [16]:
max(list(map(np.max, train_dict['t'])))

125216.59

In [43]:
list(map(len, valid_dict.values())), list(map(len, train_dict.values())), list(map(len, test_dict.values())), 

([2000, 2000], [20000, 20000], [2000, 2000])

# Print hyperparameter combinations

In [24]:
with open("/home/abhishekshar/run_hpp.sh", "r") as file:
    boilerplate = file.read()

print(boilerplate)

#!/bin/bash
#SBATCH --job-name=hpp-lvm
#SBATCH --partition m40-short
#SBATCH --mem=4096
#SBATCH --gres gpu:1
#SBATCH --output=script-out-hpp.log

. /home/abhishekshar/anaconda3/etc/profile.d/conda.sh
conda activate hpp

cd /home/abhishekshar/hierarchichal_point_process/src




In [37]:
hiddenlayers = 2**np.arange(6,11)
batch_sizes = [32,64,128]
lr = 1e-3
data_name = "retweet"
filenames = []
i = 0
for hiddenlayer in hiddenlayers:
    for batch_size in batch_sizes:
        i += 1
        print("{}\t{}\t{}".format(lr, batch_size, hiddenlayer))
        sbatch_command = "python3 main.py --max_iter=100 --model=rmtpp --data_name={} --gamma=1. --l2=1e-3 --lr={} --batch_size={} --hidden_dim={}".format(data_name, lr, batch_size, hiddenlayer)
        filename = "/home/abhishekshar/sbatch_hpp_{}_{}.sh".format(data_name, i)
        filenames.append(filename)
        print(filename)
        print(sbatch_command)
        with open(filename,"w+") as file:
            file.write(boilerplate)
            file.write(sbatch_command)
            file.write('\n')

0.001	32	64
0.001	64	64
0.001	128	64
0.001	32	128
0.001	64	128
0.001	128	128
0.001	32	256
0.001	64	256
0.001	128	256
0.001	32	512
0.001	64	512
0.001	128	512
0.001	32	1024
0.001	64	1024
0.001	128	1024


In [34]:
with open("/home/abhishekshar/{}_files".format(data_name),"w+") as file:
    for fname in filenames:
        file.write(fname)
        file.write('\n')

In [20]:
sbatch_command

'python3 main.py --max_iter=100 --model=rmtpp --data_name=meme --gamma=1. --l2=1e-3 --lr=0.001 --batch_size=128 --hidden_dim=1024'